In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

import PIL
from PIL import Image

from torch.utils.data.dataset import Dataset
from torchvision import transforms

import pandas as pd 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [5]:
data = np.loadtxt('train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


8518


In [9]:
blacks = 0
whites = 0

In [10]:
for elem in data:
    if elem[0] == 'b':
        blacks += 1
    elif elem[0] == 'w':
        whites += 1

In [41]:
blacks, whites

(1019906, 921217)

In [57]:
dataBlack = np.empty(blacks, dtype = '<U718')
dataWhite = np.empty(whites, dtype = '<U718')

In [58]:
dataBlack

array(['', '', '', ..., '', '', ''], dtype='<U718')

In [59]:
curBlack = 0
curWhite = 0

In [60]:
data.dtype

dtype('<U718')

In [61]:
for i in range(data.shape[0]):
    if data[i][0] == 'b':
        dataBlack[curBlack] = data[i]
        curBlack += 1
    elif data[i][0] == 'w':
        dataWhite[curWhite] = data[i]
        curWhite += 1

In [64]:
np.savetxt('dataBlack.txt', dataBlack, delimiter='\n', fmt="%s")

In [65]:
np.savetxt('dataWhite.txt', dataWhite, delimiter='\n', fmt="%s")

In [128]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [213]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

def toBoards(gameBad, color):
    game = gameBad.split()
    res = []
    labels = []
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, len(game)):
        curBoard = np.zeros(shape=(3, 15, 15))
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        turn = toTurn(game[i])
        if i % 2 == 1 and color == "black":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
            labels.append(turn[0] * 15 + turn[1])
        elif i % 2 == 0 and color == "white":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
            labels.append(turn[0] * 15 + turn[1])

        
        if curPlayer == 1:
            blackBoard[turn] = 1
        else:
            whiteBoard[turn] = -1


        curPlayer *= -1
        #printBoard(curBoard)
    return np.asarray(res), np.asarray(labels)

In [214]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

#### Black

In [13]:
fileSizeBlack = 1019906 // 103
print(fileSizeBlack)
fileSizeWhite = 921217 // 83
print(fileSizeWhite)

9902
11099


In [14]:
curFile = 0
for curFile in range(103):
    for gm in range(10):
        ansFeatures = np.ndarray(shape=(0, 3, 15, 15), dtype=np.int8)
        ansLabels = np.ndarray(shape=0, dtype=np.int16)
        for i in range(1000):
            res, labels = toBoards(dataBlack[curFile * fileSizeBlack + gm * 1000 + i], "black")
            ansFeatures = np.concatenate((ansFeatures, res), axis=0)
            ansLabels = np.concatenate((ansLabels, labels), axis=0)

            
        print(ansFeatures.shape)
        curStrFeatures = "poses/blackF" + str(curFile) + ".npy"
        curStrLabels = "poses/blackL" + str(curFile) + ".npy"

        np.save(curStrFeatures, ansFeatures)
        np.save(curStrLabels, ansLabels)

        curFile += 1

    

NameError: name 'toBoards' is not defined

In [15]:
class CustomDatasetFromGames(Dataset):
    def __init__(self, numBlack, numWhite=0):
        # Read data
        self.feature_arr = np.concatenate([np.load("poses/blackF" + str(i) + ".npy") for i in range(numBlack)])
        self.label_arr = np.concatenate([np.load("poses/blackL" + str(i) + ".npy") for i in range(numBlack)])
        
        # Calculate len
        self.data_len = (self.feature_arr).shape[0]
        
    def __getitem__(self, index):
        # Get ndarray from index
        feature = self.feature_arr[index]
        label = self.label_arr[index]        
        return (torch.from_numpy(feature).float(), label)

    def __len__(self):
        return self.data_len


In [16]:
datasetBlack = CustomDatasetFromGames(30)
trainLoaderBlack = torch.utils.data.DataLoader(dataset=datasetBlack, batch_size=32, shuffle=False)

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(3, affine=False)
        self.conv1 = nn.Conv2d(3, 3, 3)
        self.pad = nn.ConstantPad2d(1, 0)
        self.conv2 = nn.Conv2d(3, 2, 3)
        self.conv3 = nn.Conv2d(2, 1, 3)

    def forward(self, x):
        x = self.batchnorm(x)
        x = self.pad(F.relu(self.conv1(x)))
        x = self.pad(F.relu(self.conv2(x)))
        x = self.pad(F.relu(self.conv3(x)))

        x = x.view(-1, 225)
 
        return x

In [34]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [35]:
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0 
    
    for i, data in enumerate(trainLoaderBlack, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
print('Finished Training')

[1,   200] loss: 5.086
[1,   400] loss: 4.351
[1,   600] loss: 4.145
[1,   800] loss: 4.132
[1,  1000] loss: 4.036
[1,  1200] loss: 4.048
[1,  1400] loss: 4.004
[1,  1600] loss: 3.958
[1,  1800] loss: 3.928
[1,  2000] loss: 3.971
[1,  2200] loss: 4.025
[1,  2400] loss: 3.969
[1,  2600] loss: 3.936
[1,  2800] loss: 3.812
[1,  3000] loss: 3.753
[1,  3200] loss: 3.897
[1,  3400] loss: 3.982
[1,  3600] loss: 3.854
[1,  3800] loss: 3.948
[1,  4000] loss: 3.992
[1,  4200] loss: 4.027
[1,  4400] loss: 3.944
[1,  4600] loss: 4.004
[1,  4800] loss: 3.994
[1,  5000] loss: 4.015
[1,  5200] loss: 3.996
[1,  5400] loss: 4.003
[1,  5600] loss: 4.021
[1,  5800] loss: 4.052
[1,  6000] loss: 3.965
[1,  6200] loss: 3.918
[1,  6400] loss: 3.802
[1,  6600] loss: 3.751
[1,  6800] loss: 3.409
[1,  7000] loss: 3.794
[1,  7200] loss: 3.716
[1,  7400] loss: 3.803
[1,  7600] loss: 3.878
[1,  7800] loss: 3.841
[1,  8000] loss: 3.925
[1,  8200] loss: 3.934
[1,  8400] loss: 3.932
[1,  8600] loss: 3.940
[1,  8800] 

## Game (we are white)

In [42]:
def toTripleBoard(board, side):
    curBoard = np.zeros(shape=(3, 15, 15), dtype=np.float16)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.float16)
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.float16)

    if side == 1:
        blackTurnBoard = np.ones(shape=(15, 15), dtype=np.float16)
    else:
        blackTurnBoard = -np.ones(shape=(15, 15), dtype=np.float16)
    curBoard[0,:] = blackTurnBoard
    for i in range(15):
        for j in range(15):
            if board[i, j] == 1:
                blackBoard[i, j] = 1
            if board[i, j] == -1:
                whiteBoard[i, j] = -1
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    return curBoard


In [43]:
curBoard = np.zeros(shape=(15, 15), dtype=np.int32)

while True:
    board = toTripleBoard(curBoard, 1)
    netTurn = net(torch.from_numpy(board))
    turn = (netTurn // 15, netTurn % 15)
    curBoard[turn] = 1
    for i in range(15):
        for j in range(15):
            if curBoard[i, j] == -1:
                print(-1, end="")
            else:
                print(curBoard[i, j], end=" ")
        print()
    myTurn = list(map(int, input().split()))
    curBoard[myTurn] = -1
    for i in range(15):
        for j in range(15):
            if curBoard[i, j] == -1:
                print(-1, end="")
            else:
                print(curBoard[i, j], end=" ")
        print()

ValueError: expected 4D input (got 3D input)